## ElasticSearch Requests

Jay Urbain, PhD
jay.urbain@gmail.com


Simple test query for local ES server

In [18]:
streets = 'east standish place and lake drive'
data = '{"query" : {"match": { "streets": "' + streets + '" }}, "highlight" : {"fields" : {"streets" : {}}}}'

headers={'Accept': 'application/json', 'Content-type': 'application/json'}

elastic_url ='http://localhost:9200/intersections_p2/_search'

#query = json.dumps(json_data)

#response = requests.get(elastic_url, data = query, auth=('xxx','xxx'), verify=False, headers = headers)

response = requests.get(elastic_url, data=data, headers = headers)

print( type(response.text))

print(response.text)

<class 'str'>
{"took":23,"timed_out":false,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0},"hits":{"total":{"value":109,"relation":"eq"},"max_score":14.05933,"hits":[{"_index":"intersections_p2","_type":"_doc","_id":"196635218","_score":14.05933,"_source":{"location": {"lat": 43.1796597, "lon": -87.8952999}, "streets": ["North Lake Drive", "East Standish Place"], "place": "Bayside, Wisconsin"},"highlight":{"streets":["North <em>Lake</em> <em>Drive</em>","<em>East</em> <em>Standish</em> <em>Place</em>"]}},{"_index":"intersections_p2","_type":"_doc","_id":"196665007","_score":10.578792,"_source":{"location": {"lat": 43.179363, "lon": -87.8912343}, "streets": ["East Standish Place", "North Bayside Drive"], "place": "Bayside, Wisconsin"},"highlight":{"streets":["<em>East</em> <em>Standish</em> <em>Place</em>","North Bayside <em>Drive</em>"]}},{"_index":"intersections_p2","_type":"_doc","_id":"196665005","_score":10.578792,"_source":{"location": {"lat": 43.1795345, "lon": -87.89

Figure out how to process search results

In [24]:
import json

d = json.loads(response.text)
hits = d['hits']['hits']

for h in hits:
    score = h['_score']
    source = h['_source']['streets']
    place = h['_source']['place']
    highlight = h['highlight']
    print(score, source, highlight, place)

14.05933 ['North Lake Drive', 'East Standish Place'] {'streets': ['North <em>Lake</em> <em>Drive</em>', '<em>East</em> <em>Standish</em> <em>Place</em>']} Bayside, Wisconsin
10.578792 ['East Standish Place', 'North Bayside Drive'] {'streets': ['<em>East</em> <em>Standish</em> <em>Place</em>', 'North Bayside <em>Drive</em>']} Bayside, Wisconsin
10.578792 ['North Tennyson Drive', 'East Standish Place'] {'streets': ['North Tennyson <em>Drive</em>', '<em>East</em> <em>Standish</em> <em>Place</em>']} Bayside, Wisconsin
10.578792 ['North Tennyson Drive', 'East Standish Place'] {'streets': ['North Tennyson <em>Drive</em>', '<em>East</em> <em>Standish</em> <em>Place</em>']} Bayside, Wisconsin
9.2472925 ['East Standish Place'] {'streets': ['<em>East</em> <em>Standish</em> <em>Place</em>']} Bayside, Wisconsin
9.051197 ['North Lake Drive', 'East Buttles Place'] {'streets': ['North <em>Lake</em> <em>Drive</em>', '<em>East</em> Buttles <em>Place</em>']} Bayside, Wisconsin
9.051197 ['North Lake Driv

Allstate sample, deidentified with dubbed locations for Bayside Wisconsin

In [2]:
sample = "plus ok purpose trip time accident going coming think leaving friend's house i'm going   actually going parents house friend's house headed south tennyson drive towards hermitage runs east west heading east light light like two turn lane left wait left green arrow light uh light green sunny cloud sky ok weather clear raining ok traffic light moderate heavy light ok passengers vehicle time right what's name road happened um accident actually happened clark road know ok here's ok here's weird thing clark road one point turns road turns stickney point i'm sure stickney point clark road clock that's fine that's fine road coming making left turn oh headed east start heading south swift road turning east onto clark golf gate estates florida um yeah left turn swift o'clock road correct right said outside light green green blew red light solid green already completed turn hit well paula green light arrow green light understand need verify made left turn solid green light arrow green light oh i'm firms arrow green light arrow ok made left turn arrow green light first intersection waiting light turn green ok first intersection ok made left road going straight heading east uh blew clark swift light said heading east ok yes remember heading south swift east   ok police involved oh yeah called report needed fault yep gave ticket reckless driving"

Method to extract street names from ES highlighted results.

```
Takes list of strings of the form:   
    North <em>Tennyson</em> <em>Drive</em>

Returns list of strings of the form:   
    tennyson drive
```

In [45]:
import re

def extract_street_from_highlight(li):
    '''
    li - input list of strings: North <em>Tennyson</em> <em>Drive</em>
    lo - output list of strings: tennyson drive
    '''
    lo = []
    for i in li:
        for s in i[2]['streets']:
            print( s )
            street_search = re.search('<em>(.*)</em>', s, re.IGNORECASE)
            street = street_search.group(1)
            street = street.replace('<em>', '').replace('</em>','')
            lo.append(street.lower())
            prinit(street.lower())
            return lo

Method to search street names given:  
- `elastic_url` - elasticsearch url
- `streets` - query, for example, "blah blah Tennyson Drive and East Standish Place blah blah blah"
- `place` - filter by the place associated with streets when indexed (optional)

In [41]:
import json
import requests
import re

def search_streets(elastic_url, streets, place=''):
    
    data = '{"query" : {"match": { "streets": "' + streets + '" }}, "highlight" : {"fields" : {"streets" : {}}}}'
    
    data = '''{{
        "query": {{
            "bool": {{
                "must": [
                    {{
                        "match": {{
                            "streets": "{0}"
                        }}
                    }},
                    {{
                        "match": {{
                            "place": "{1}"
                        }}
                    }}
                ]
            }}
        }},
        "highlight": {{
            "fields": {{
                "streets": {{}}
            }}
        }}
    }}'''.format('tennyson drive', 'bayside')
    
    print(data)

    headers={'Accept': 'application/json', 'Content-type': 'application/json'}

    response = requests.get(elastic_url, data=data, headers = headers)
    d = json.loads(response.text)
    hits = d['hits']['hits']

    d = json.loads(response.text)
    street_list = []
    for h in hits:
        score = h['_score']
        source = h['_source']['streets']
        place = h['_source']['place']
        highlight = h['highlight']
        street_list.append( (score, source, highlight, place) )
    return street_list
        
# elastic_url = 'http://localhost:9200/intersections_p2/_search'
elastic_url = 'https://search-intersections-p2-zoxb5qvz4i4en6vp5pg7yv44vy.us-east-1.es.amazonaws.com/intersections-p2/_search'
streets = sample

li = search_streets(elastic_url, streets)
    
for i in li:
    print(i)

{
        "query": {
            "bool": {
                "must": [
                    {
                        "match": {
                            "streets": "tennyson drive"
                        }
                    },
                    {
                        "match": {
                            "place": "bayside"
                        }
                    }
                ]
            }
        },
        "highlight": {
            "fields": {
                "streets": {}
            }
        }
    }
(26.18042, ['North Tennyson Drive', 'East Standish Place'], {'streets': ['North <em>Tennyson</em> <em>Drive</em>']}, 'Bayside, Wisconsin')
(25.868534, ['East Bay Point Road', 'North Tennyson Drive'], {'streets': ['North <em>Tennyson</em> <em>Drive</em>']}, 'Bayside, Wisconsin')
(25.346392, ['North Tennyson Drive', 'East Manor Circle'], {'streets': ['North <em>Tennyson</em> <em>Drive</em>']}, 'Bayside, Wisconsin')
(25.22023, ['North Tennyson Drive', 'East Standish

In [42]:
import re

for i in li:
    for s in i[2]['streets']:
        print( s )
        street_search = re.search('<em>(.*)</em>', s, re.IGNORECASE)
        street = street_search.group(1)
        street = street.replace('<em>', '').replace('</em>','')
        print(street.lower())
        
        print( sample.find(street.lower()) )


North <em>Tennyson</em> <em>Drive</em>
tennyson drive
146
North <em>Tennyson</em> <em>Drive</em>
tennyson drive
146
North <em>Tennyson</em> <em>Drive</em>
tennyson drive
146
North <em>Tennyson</em> <em>Drive</em>
tennyson drive
146
North <em>Tennyson</em> <em>Drive</em>
tennyson drive
146
North <em>Tennyson</em> <em>Drive</em>
tennyson drive
146
North <em>Tennyson</em> <em>Drive</em>
tennyson drive
146
North Waverly <em>Drive</em>
drive
155
North Fairway <em>Drive</em>
drive
155
North Lake <em>Drive</em>
drive
155
North Lake <em>Drive</em>
drive
155


Bulk data insert

In [55]:
def post_bulk_insert(endpoint, data):
   response = requests.post(endpoint, data=data, headers={'content-type':'application/json', 'charset':'UTF-8'})
   return response

In [56]:
!pwd

/Users/jayurbain/Dropbox/SearchStreetNames/SearchStreetNames


In [ ]:
import pandas as pd
import numpy as np

endpoint = 'https://search-intersections-p2-zoxb5qvz4i4en6vp5pg7yv44vy.us-east-1.es.amazonaws.com/intersections-p2/_bulk'
datafile = 'florida_intersections-p2.json'

with open(datafile) as fp:
    line1 = fp.readline()
    line2 = fp.readline()
    data = line1 + line2
    print(data)
    post_bulk_insert(endpoint, data)
    
    cnt = 2
    while line1 and line2:
        line1 = fp.readline()
        line2 = fp.readline()
        data = line1 + line2
#         print(data)
        post_bulk_insert(endpoint, data)
        cnt += 2
    print(str(cnt), 'records inserted')

{"index": {"_index": "intersections-p2", "_type": "_doc", "_id": 99094596}}
{"location": {"lat": 24.5474923, "lon": -81.7892392}, "streets": ["Casa Marina Court", "Reynolds Street", "Atlantic Boulevard"], "place": "Florida"}

